In [4]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 90

In [5]:
# Get our block frost api key from the file
with open('keys.txt', 'r') as file:
    api_key = file.read().replace('\n', '')
api_key = api_key.split('=')[1]

In [6]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls

api = BlockFrostApi(
	project_id=api_key,
	base_url=ApiUrls.mainnet.value,
)

In [7]:
try:
    health = api.health()
    print(health)   # prints object:    HealthResponse(is_healthy=True)
    health = api.health(return_type='json') # Can be useful if python wrapper is behind api version
    print(health)   # prints json:      {"is_healthy":True}
    health = api.health(return_type='pandas')
    print(health)   # prints Dataframe:         is_healthy
                    #                       0         True

    
    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        return_type='pandas'
    )
    print(account_rewards)  # prints 221
    print(len(account_rewards))  # prints 20

    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        gather_pages=True, # will collect all pages
    )
    print(account_rewards[0].epoch)  # prints 221
    print(len(account_rewards))  # prints 57

    address = api.address(
        address='addr1qxqs59lphg8g6qndelq8xwqn60ag3aeyfcp33c2kdp46a09re5df3pzwwmyq946axfcejy5n4x0y99wqpgtp2gd0k09qsgy6pz')
    print(address.type)  # prints 'shelley'
    for amount in address.amount:
        print(amount.unit)  # prints 'lovelace'

except ApiError as e:
    print(e)

Namespace(is_healthy=True)
{'is_healthy': True}
   is_healthy
0        True
    epoch    amount                                            pool_id
0     221  16576013  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
1     222  17132278  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
2     223  15649019  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
3     224  17281719  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
4     225  18604433  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
5     226  12591619  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
6     227  17166845  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
7     228  17391311  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
8     231  13442322  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
9     234   4412912  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
10    235   5175375  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
11    236   8040874  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
1

# This code below is used to get all the Pool_ids (bech32 format) for all registered pools on Cardano

In [8]:
df = pd.DataFrame()

for i in range(1,100):
        try:
                pool_ids = api.pools(
                        gather_pages=True,
                        count=100,
                        page=i,)
                if len(pool_ids)==0:
                        break
                pools_list = pd.DataFrame(pool_ids)
                df = df.append(pools_list)
        except ApiError as e:
                print(e)


In [9]:
df

,0
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...
4,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...
...,...
27,pool1mnuwy85gq5wvl426dwh4zz5laryatvrvh6sw8dlvq...
28,pool14glhqr7989jz4yu0hnytpw62xlq0666sy075ztufd...
29,pool1ttxtgplmy2v7t9s56gm3xxhsyxe68kpx0vu3utts0...
30,pool1jj674cqm2uaexhwjkuv23nav6egcwz9f5xls76nfh...


## Now let's get the list of retiring pools and then we will cross reference it with the list of all the pools and then remove any retiring pools

In [10]:
df2 = pd.DataFrame()

for i in range(1,50):
        try:
                retired_pools = api.pools_retiring(gather_pages=True, count=100, page=i)
                pools = pd.DataFrame(retired_pools)
                df2 = df2.append(pools)
        except ApiError as e:
                print(e)

In [11]:
df2

,0
0,Namespace(pool_id='pool1a3u8zuz7fqavxgl48s8k37...
1,Namespace(pool_id='pool14zx5y80c5m5x5hch6e57pc...
2,Namespace(pool_id='pool15eera5yr3eur7tz067wfpp...
3,Namespace(pool_id='pool1wqzdqsm02tust35dds3u7k...
4,Namespace(pool_id='pool1qgm942xc95w8f6qsx2vd35...
5,Namespace(pool_id='pool1ffuh4tz68rmv9tcdfqazlu...
6,Namespace(pool_id='pool1hkcy0k7fjlcwtuyh6wsdn2...
7,Namespace(pool_id='pool1pg9twqvv0e9ds3q4n9dcd9...


In [15]:
df.iloc[0,0]

'pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087fdld7yws0xt'

In [24]:
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()

for i in range(len(df)):
        try:
                pool_info = api.pool(pool_id=df.iloc[i,0], return_type='json')
                df3 = df3.append(pool_info, ignore_index=True)
                time.sleep(0.10)
        except ApiError as e:
                print(e)
        

In [22]:
df3

,pool_id,hex,vrf_key,blocks_minted,live_stake,live_size,live_saturation,live_delegators,active_stake,active_size,declared_pledge,live_pledge,margin_cost,fixed_cost,reward_account,owners,registration,retirement
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,0220a5d08adbfe9554b52d7b2993be5892ac3ff340e674...,3953.0,32717971823697,1.376010e-03,4.893655e-01,3201.0,33517632780856,1.406643e-03,100000000000,571490748583,0.009,340000000,stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp...,[stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edj...,[841cca81da918feb9fa7257a34630eac95794be712ed3...,[]
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,b512cc7c1a8ba689c2d8fd27adfdbac2049a3f8f95c8b8...,1720.0,19439668801706,8.175682e-04,2.907608e-01,167.0,19433780729967,8.155824e-04,510000000000,560639019207,0.049,340000000,stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389ps...,[stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389p...,[a96c79773b7506211eb56bf94886a2face17657d1009f...,[]
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,b8ba5f7dcbf17e95781e325114da46def1fa908435a27d...,74.0,555391415162,2.335793e-05,8.307037e-03,15.0,557889484549,2.341309e-05,50100000000,550071618613,0.000,340000000,stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdxr...,[stake1uyx0ut5wc5nmu0tacjc9pyyahdyq5ujghwljkdx...,[37f5682e2622c821175bab9d5637b7321ecacc5ec6636...,[]
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,01df29429173d263c7533a22742dae19f16a08798b7a57...,b75115a1e4e32a33985fc720f3715f0e99617b448fbc36...,252.0,1837715988715,7.728826e-05,2.748687e-02,719.0,1749933936083,7.343992e-05,200000000000,205098253999,0.005,340000000,stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0me...,[stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0m...,[e8951eddb19b697bc74549aa9d4f956a8be8d5cf2c7a8...,[]
4,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,78b0b08fec7458afd1209dcd26bd0c10faa640a49d5013...,23.0,4000924534,1.682657e-07,5.984217e-05,14.0,4000924534,1.679078e-07,7149000000,0,0.050,340000000,stake1u9w20uzsevkxe6vp2nlmqjjrcj4gvcf560tvufuv...,[stake1u9w20uzsevkxe6vp2nlmqjjrcj4gvcf560tvufu...,[ff2d9576bd9c5e1dff6c9987860de39366f7b75dd6bbc...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,pool1mnuwy85gq5wvl426dwh4zz5laryatvrvh6sw8dlvq...,dcf8e21e88051ccfd55a6baf510a9fe8c9d5b06cbea0e3...,4410782ff4b0be7603c8b3d2b6d56b059abad8267082ad...,0.0,7622534,3.205779e-10,1.140109e-07,1.0,0,0.000000e+00,0,7622534,0.020,340000000,stake1u85kvwgadv90ngx7per6ve4xr0khsprf0j9406tr...,[stake1u85kvwgadv90ngx7per6ve4xr0khsprf0j9406t...,[6b50b1debcdfb960c435eeb929c3c96cc2013d2c234f0...,[]
3128,pool14glhqr7989jz4yu0hnytpw62xlq0666sy075ztufd...,aa3f700fc539642a938fbcc8b0bb4a37c0fd6b5023fd41...,c59898f9ab50ca44eb872ab3c82478d1b6ff37742a3011...,0.0,18576493690,7.812642e-07,2.778502e-04,14.0,0,0.000000e+00,1000000000,1551880186,0.010,340000000,stake1ux5x2rxrz0gtm47l0lmlweze78hrfg9gfusx7a49...,[stake1uy944d4rk80fv9s96kj8d30puqnzsk96hzg3fq0...,[c6ae472813e8daa9ab17e0f228be250a81bfd4ec3fada...,[]
3129,pool1ttxtgplmy2v7t9s56gm3xxhsyxe68kpx0vu3utts0...,5accb407fb2299e59614d237131af021b3a3d8267b391e...,ff42aeff2268e409ae66f07536614c2606f16057060d99...,0.0,12740495408914,5.358219e-04,1.905607e-01,590.0,0,0.000000e+00,1000000000,1163618266,0.990,350000000,stake1u92shztzmuwjwemkk8c0vnarshw6c4k09rp8znuy...,[stake1u92shztzmuwjwemkk8c0vnarshw6c4k09rp8znu...,[156cfc963e6591e346db9ea98c1760e9ad309aa1537fb...,[]
3130,pool1jj674cqm2uaexhwjkuv23nav6egcwz9f5xls76nfh...,94b5eae01b573b935dd2b718a8cfacd6518708a9a1bf0f...,ac066e21c6f93a732ce16d99059f0b71b5aef41e0f0a81...,0.0,204547285,8.602563e-09,3.059431e-06,1.0,0,0.000000e+00,100000000,204547285,0.050,340000000,stake1u9fxxtladkxfepsxxnpnc0p4q6z962387e5qw444...,[stake1u9fxxtladkxfepsxxnpnc0p4q6z962387e5qw44...,[a3af27275bf2ac352edc0cefc0d6d330a766e117d

In [29]:
relays_info = api.pool_relays(pool_id=df3.pool_id[0],return_type='pandas')
relays_info

,ipv4,ipv6,dns,dns_srv,port
0,54.220.20.40,None,None,None,3002
